# <div class="title_h1">Introduction to linear regression</div>

## <div class="title_h2">Batter up</div>

The movie [Moneyball](https://en.wikipedia.org/wiki/Moneyball_(film)) focuses on the "quest for the secret of success in baseball". It follows a low-budget team, the Oakland Athletics, who believed that underused statistics, such as a player's ability to get on base, betterpredict the ability to score runs than typical statistics like home runs, RBIs (runs batted in), and batting average. Obtaining players who excelled in these underused statistics turned out to be much more affordable for the team.

In this lab we'll be looking at data from all 30 Major League Baseball teams and examining the linear relationship between runs scored in a season and a number of other player statistics. Our aim will be to summarize these relationships both graphically and numerically in order to find which variable, if any, helps us best predict a team's runs scored in a season.

## <div class="title_h2">The data</div>

Let's load up the data for the 2011 season.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import io
import requests

df_url = 'https://raw.githubusercontent.com/imranture/datasets/master/mlb11.csv'
url_content = requests.get(df_url, verify = False).content
mlb11 = pd.read_csv(io.StringIO(url_content.decode('utf-8')))

mlb11

In addition to runs scored, there are seven traditionally used variables in the data set: at-bats, hits, home runs, batting average, strikeouts, stolen bases, and wins. There are also three newer variables: on-base percentage, slugging percentage, and on-base plus slugging. For the first portion of the analysis we'll consider the seven traditional variables. At the end of the lab, you'll work with the newer variables on your own.

<div class = 'exercise'>
<h4>Exercise 1</h4>
What type of plot would you use to display the relationship between <code>runs</code> and one of the other numerical variables? Plot this relationship using the variable <code>at_bats</code> as the predictor. Does the relationship look linear? If you knew a team's <code>at_bats</code>, would you be comfortable using a linear model to predict the number of runs?
</div>

If the relationship looks linear, we can quantify the strength of the relationship with the correlation coefficient.

In [ ]:
mlb11['runs'].corr(mlb11['at_bats'])

## <div class="title_h2">Sum of squared residuals</div>

Think back to the way that we described the distribution of a single variable. Recall that we discussed characteristics such as center, spread, and shape. It's also useful to be able to describe the relationship of two numerical variables, such as `runs` and `at_bats` above.

<div class = 'exercise'>
<h4>Exercise 2</h4>
Looking at your plot from the previous exercise, describe the relationship between these two variables. Make sure to discuss the form, direction, and strength of the relationship as well as any unusual observations.
</div>

Recall that the difference between the observed values and the values predicted by the line are called *residuals*. Note that the data set has 30 observations in total, hence there are 30 residuals.

#### ${e}$<sub>i</sub> = ${y}$<sub>i</sub>−$\bar{y}$<sub>i</sub>

To visualize the residuals of a linear regression, we can use `residplot()` function from `seaborn`:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (10,5)

sns.residplot(data = mlb11, x = 'at_bats', y = 'runs', color = 'red')
plt.show();

## <div class="title_h2">The linear model</div>

In order to determine the best fit line we can use `statsmodel', a very useful module for the estimation of many different statistical models, as well as for conducting statistical tests, and statistical data exploration.

In [ ]:
import statsmodels.api as sm

formula_string = "runs ~ at_bats"

model = sm.formula.ols(formula = formula_string, data = mlb11)
model_fitted = model.fit()

print(model_fitted.summary())

Let's print the intercept and slope values.

In [ ]:
print('Intercept =', model_fitted.params[0])
print('Slope =', model_fitted.params[1])

Knowing the intercept and slope, we can write down the least squares regression line for the linear model:

#### ${y}$ = - 2789.2429 + 0.6305 x ${atbats}$

One last piece of information we will discuss from the summary output is the Multiple R-squared, or more simply, ${R}$<sup>2</sup>. The ${R}$<sup>2</sup> value represents the proportion of variability in the response variable that is explained by the explanatory variable. For this model, 37.3% of the variability in runs is explained by at-bats.

In [ ]:
print('R-squared =', model_fitted.rsquared)

<div class = 'exercise'>
<h4>Exercise 3</h4>
    Fit a new model that uses <code>homeruns</code> to predict <code>runs</code>. Using the estimates from the Python output, write the equation of the regression line. What does the slope tell us in the context of the relationship between success of a team and its home runs?
</div>

## <div class="title_h2">Prediction and prediction errors</div>

Just as we used the mean and standard deviation to summarize a single variable, we can summarize the relationship between these two variables by finding the line that best follows their association. Let's plot `at_bats` and `runs` on a scatter plot.

In [ ]:
x = mlb11['at_bats']
y = mlb11['runs']

y_pred = model_fitted.predict(x)

plt.scatter(mlb11['at_bats'], mlb11['runs'])
plt.plot(x, y_pred, color = 'orange')
plt.show();

<div class = 'exercise'>
<h4>Exercise 4</h4>
If a team manager saw the least squares regression line and not the actual data, how many runs would he or she predict for a team with 5,578 at-bats? Is this an overestimate or an underestimate, and by how much? In other words, what is the residual for this prediction?
</div>

## <div class="title_h2">Model diagnostics</div>

To assess whether the linear model is reliable, we need to check for (1) linearity, (2) nearly normal residuals, and (3) constant variability.

*Linearity*: You already checked if the relationship between runs and at-bats is linear using a scatterplot. We should also verify this condition with a plot of the residuals vs. at-bats.

In [ ]:
import seaborn as sns
sns.residplot(mlb11, x = 'at_bats', y = 'runs', color = 'red')
plt.xlabel('at_bats', fontsize = 16)
plt.ylabel('residuals', fontsize = 16)
plt.show();

<div class = 'exercise'>
<h4>Exercise 5</h4>
Is there any apparent pattern in the residuals plot? What does this indicate about the linearity of the relationship between runs and at-bats?
</div>

*Nearly normal residuals*: To check this condition, we can look at a histogram.

In [ ]:
residuals = (y - y_pred)
plt.hist(residuals, bins = 8)
plt.show();

or a normal probability plot of the residuals.

In [ ]:
from scipy.stats import probplot
probplot(residuals, plot = plt)
plt.show(); 

<div class = 'exercise'>
<h4>Exercise 6</h4>
Based on the histogram and the normal probability plot, does the nearly normal residuals condition appear to be met?</div>

*Constant variability*:

<div class = 'exercise'>
<h4>Exercise 7</h4>
Based on the plot in (1), does the constant variability condition appear to be met?
    </div>

---
### <div class="title_h3">On Your Own</div>

<ol>
    <li>Choose another traditional variable from <code>mlb11</code> that you think might be a good predictor of <code>runs</code>. Produce a scatterplot of the two variables and fit a linear model. At a glance, does there seem to be a linear relationship?</li><br>
    <li>How does this relationship compare to the relationship between <code>runs</code> and <code>at_bats</code>? Use the R squared values from the two model summaries to compare. Does your variable seem to predict <code>runs</code> better than <code>at_bats</code>? How can you tell?</li><br>
    <li>Now that you can summarize the linear relationship between two variables, investigate the relationships between <code>runs</code> and each of the other five traditional variables. Which variable best predicts <code>runs</code>? Support your conclusion using the graphical and numerical methods we've discussed (for the sake of conciseness, only include output for the best variable, not all five).</li><br>
    <li>Now examine the three newer variables. These are the statistics used by the author of <i>Moneyball</i> to predict a teams success. In general, are they more or less effective at predicting runs that the old variables? Explain using appropriate graphical and numerical evidence. Of all ten variables we've analyzed, which seems to be the best predictor of <code>runs</code>? Using the limited (or not so limited) information you know about these baseball statistics, does your result make sense?</li><br>
    <li>Check the model diagnostics for the regression model with the variable you decided was the best predictor for <code>runs</code></li>
</ol>

***

<div class = "license">
    This lab was adapted by <b>Vural Aksakalli</b> and <b>Imran Ture</b> from OpenIntro by Andrew Bray and Mine Çetinkaya-Rundel.
</div>